### ESTUDO DE CASO - TESTE A/B

In [1]:
# Watermark para gravar a versão dos pacotes no jupyter notebook
!pip install -q -U watermark


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importa pacotes
import pandas as pd
import numpy as np

In [3]:
# Versões dos pacotes usados nesse jupyter notebook
%reload_ext watermark
%watermark -a "Maria Eduarda" --iversions

Author: Maria Eduarda

pandas: 1.2.3
sys   : 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
numpy : 1.20.2



#### Análise Exploratória

version = Se o jogador foi colocado em um portão de nível 30 ou 40

sumgamerounds = Número de rodadas jogadas por jogador durante os 14 primeiros dias depois da instalação

retention_1 = Se o jogador voltou para jogar depois de 1 dia de instalação

retention_7 = Se o jogador voltou para jogar depois de 7 dias de instalação

In [6]:
# fonte: https://www.kaggle.com/datasets/mursideyarkin/mobile-games-ab-testing-cookie-cats?resource=download
dados = pd.read_csv('cookie_cats.csv')
dados

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [7]:
dados.shape

(90189, 5)

In [8]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [9]:
dados.isnull().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

In [10]:
# Proporção de retenção em 1 dia
dados.retention_1.value_counts()

False    50036
True     40153
Name: retention_1, dtype: int64

In [11]:
# Proporçaõ de retenção em 7 dias
dados.retention_7.value_counts()

False    73408
True     16781
Name: retention_7, dtype: int64

#### Cálculo de probabilidades básicas
Supondo que:

Grupo de controle (o que já tem) = gate_30 

Grupo de teste/tratamento = gate_40

In [12]:
# Probabilidade de um usuário visualizar o gate_30
dados[dados.version == 'gate_30'].shape[0]/dados.shape[0] * 100

49.56258523766757

In [13]:
# Probabilidade de um usuário visualizar o gate_40
dados[dados.version == 'gate_40'].shape[0]/dados.shape[0] * 100

50.43741476233243

In [14]:
dados.shape

(90189, 5)

In [15]:
# Substitui dados usando função

def substitui(x):
    if x == 'False':
        x = 0
    else:
        x = 1
    return x

dados['retention_1_num'] = dados['retention_1'].astype(str).apply(substitui)   
dados['retention_7_num'] = dados['retention_7'].astype(str).apply(substitui)
dados


,userid,version,sum_gamerounds,retention_1,retention_7,retention_1_num,retention_7_num
0,116,gate_30,3,False,False,0,0
1,337,gate_30,38,True,False,1,0
2,377,gate_40,165,True,False,1,0
3,483,gate_40,1,False,False,0,0
4,488,gate_40,179,True,True,1,1
...,...,...,...,...,...,...,...
90184,9999441,gate_40,97,True,False,1,0
90185,9999479,gate_40,30,False,False,0,0
90186,9999710,gate_30,28,True,False,1,0
90187,9999768,gate_40,51,True,False,1,0


### Criação da Linha de Base (Baseline)

#### Retenção 1 dia

In [16]:
# Escolhe apenas 2 colunas do dataset
df_ab_retencao_1  = dados[['version','retention_1']]
#df_ab_retencao_1.columns = [['versao','retencao_1']] # forma A de renomeação de colunas ERRO EM PIVOT_TABLE
df_ab_retencao_1.rename(columns={'version':'versao','retention_1':'retencao_1'}, inplace=True) # forma B de renomeação de colunas
df_ab_retencao_1.head()

c:\Users\madu_\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,versao,retencao_1
0,gate_30,False
1,gate_30,True
2,gate_40,True
3,gate_40,False
4,gate_40,True


In [17]:
df_ab_retencao_1.versao.value_counts()

gate_40    45489
gate_30    44700
Name: versao, dtype: int64

In [18]:
df_ab_retencao_1.shape

(90189, 2)

In [19]:
dados_sumario1 = df_ab_retencao_1.pivot_table(values='retencao_1', index='versao', aggfunc=np.sum)
dados_sumario1

,retencao_1
versao,
gate_30,20034
gate_40,20119


In [20]:
# Somatório de linhas
dados_sumario1['total'] = df_ab_retencao_1.pivot_table(values='retencao_1', index='versao', aggfunc = lambda x: len(x))
dados_sumario1['total']

versao
gate_30    44700
gate_40    45489
Name: total, dtype: int64

In [21]:
# Sumário com taxa em porcentagem
dados_sumario1['taxa'] = df_ab_retencao_1.pivot_table(values='retencao_1', index='versao')
dados_sumario1['taxa']

versao
gate_30    0.448188
gate_40    0.442283
Name: taxa, dtype: float64

In [22]:
dados_sumario1

,retencao_1,total,taxa
versao,,,
gate_30,20034,44700,0.448188
gate_40,20119,45489,0.442283


In [23]:
# Conferindo taxa de conversão
df_ab_retencao_1.loc[(df_ab_retencao_1['versao'] == 'gate_40') & (df_ab_retencao_1['retencao_1'] == True)]

,versao,retencao_1
2,gate_40,True
4,gate_40,True
5,gate_40,True
8,gate_40,True
9,gate_40,True
...,...,...
90150,gate_40,True
90159,gate_40,True
90181,gate_40,True
90184,gate_40,True


In [24]:
# Obtem os valores de gate_30

conversao1_30 = dados_sumario1['retencao_1'][0]
print('conversao_30 =',conversao1_30)
total1_30 = dados_sumario1['total'][0]
print('total_30 = ', total1_30)
taxa1_30 = dados_sumario1['taxa'][0]
print('taxa_30 =', taxa1_30)

conversao_30 = 20034
total_30 =  44700
taxa_30 = 0.4481879194630872


In [25]:
# Obtem os valores de gate_40

conversao1_40 = dados_sumario1['retencao_1'][1]
print('conversao_40 =',conversao1_40)
total1_40 = dados_sumario1['total'][1]
print('total_40 = ', total1_40)
taxa1_40 = dados_sumario1['taxa'][1]
print('taxa_40 =', taxa1_40)

conversao_40 = 20119
total_40 =  45489
taxa_40 = 0.44228274967574577


#### Retenção 7 dias

In [26]:
# Escolhe apenas 2 colunas do dataset
df_ab_retencao_7 = dados[['version','retention_7']]
df_ab_retencao_7.rename(columns={'version':'versao', 'retention_7':'retencao_7'}, inplace=True)
df_ab_retencao_7


c:\Users\madu_\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,versao,retencao_7
0,gate_30,False
1,gate_30,False
2,gate_40,False
3,gate_40,False
4,gate_40,True
...,...,...
90184,gate_40,False
90185,gate_40,False
90186,gate_30,False
90187,gate_40,False


In [27]:
df_ab_retencao_7.versao.value_counts()

gate_40    45489
gate_30    44700
Name: versao, dtype: int64

In [28]:
df_ab_retencao_7.shape

(90189, 2)

In [29]:
dados_sumario7 = df_ab_retencao_7.pivot_table(values='retencao_7', index='versao', aggfunc=np.sum)
dados_sumario7

,retencao_7
versao,
gate_30,8502
gate_40,8279


In [30]:
# Confirmando os valores da célula anterior
df_ab_retencao_7.loc[(df_ab_retencao_7['versao'] == 'gate_40') & (df_ab_retencao_7['retencao_7'] == True)]

,versao,retencao_7
4,gate_40,True
5,gate_40,True
8,gate_40,True
10,gate_40,True
27,gate_40,True
...,...,...
90059,gate_40,True
90115,gate_40,True
90125,gate_40,True
90150,gate_40,True


In [31]:
dados_sumario7['total'] = df_ab_retencao_7.pivot_table(values='retencao_7', index='versao', aggfunc= lambda x: len(x))
dados_sumario7 

,retencao_7,total
versao,,
gate_30,8502,44700
gate_40,8279,45489


In [32]:
dados_sumario7['taxa'] = df_ab_retencao_7.pivot_table(values='retencao_7', index='versao')
dados_sumario7

,retencao_7,total,taxa
versao,,,
gate_30,8502,44700,0.190201
gate_40,8279,45489,0.182000


In [33]:
# Obtem os valores de gate_30

conversao7_30 = dados_sumario7['retencao_7'][0]
print('conversao7_30 =',conversao7_30)
total7_30 = dados_sumario7['total'][0]
print('total_30 = ', total7_30)
taxa7_30 = dados_sumario7['taxa'][0]
print('taxa_30 =', taxa7_30)

conversao7_30 = 8502
total_30 =  44700
taxa_30 = 0.19020134228187918


In [34]:
# Obtem os valores de gate_40

conversao7_40 = dados_sumario7['retencao_7'][1]
print('conversao7_40 =',conversao7_40)
total7_40 = dados_sumario7['total'][1]
print('total_40 = ', total7_40)
taxa7_40 = dados_sumario7['taxa'][1]
print('taxa_40 =', taxa7_40)

conversao7_40 = 8279
total_40 =  45489
taxa_40 = 0.18200004396667327


### Execução do Teste de Hipótese

Executamos o teste de hipótese e registramos a taxa de sucesso de cada grupo

Poder estatístico ou sensibilidade

Igual a 1 - Beta

Normalmente 80% é usada para a maioria das análsies. É a probalbilidade de rejeitar a hipótese nula quando a hipótese nula é de fato falsa

Parâmetros para o teste:

1 - Alfa (nível de significância): normalmente 5%; probabilidade de rejeitar a hipótese nula quando a hipótese nula for verdadeira

2 - Beta: probabilidade de aceitar a hipótese nula quando a hipótese nula é realmente falsa



In [35]:
# Parâmetros para o teste

alfa = 0.05
beta = 0.2

#### Utilizando Teste Chi-Quadrado

##### scipy.chisquare

Usei o teste chi-quadrado com o comando stats.chisquare porque ele é apropriado para verificar se uma distribuição binomial é consistente com uma hipótese específica. O teste chi-quadrado é usado para comparar uma distribuição de frequência esperada com uma distribuição de frequência observada. No caso, a hipótese nula é que as distribuições binomiais dos grupos A e B são iguais e a hipótese alternativa é que elas são diferentes. O teste chi-quadrado nos permite determinar se as diferenças entre as distribuições observadas e esperadas são estatisticamente significativas ou se podem ser explicadas por acaso.

No exemplo seguinte, a distribuição de frequência esperada é calculada com base na probabilidade de sucesso (p) de cada grupo (A e B) e no tamanho da amostra (n) de cada grupo. A distribuição de frequência observada é o número de sucessos (k) registrados em cada grupo.

A função scipy.chisquare é usada para realizar um teste de hipótese para verificar se uma amostra de dados segue uma distribuição específica. Ela aceita como entrada os dados observados e a distribuição teórica esperada, e retorna o valor do estatístico de teste chi-square e o p-valor. 

n_A = 44700
p_A = 0.4481
f_exp_A = n_A*p_A

E a distribuição de frequência observada é k_A = 20034

In [36]:
from scipy.stats import binom, chisquare

# Dados do grupo A
n_A = total1_30
k_A = conversao1_30
p_A = taxa1_30

# Dados do grupo B
n_B = total1_40
k_B = conversao1_40
p_B = taxa1_40

# Cálculo do teste chi-quadrado
stat, p_value = chisquare(f_obs=[k_A, k_B], f_exp=[n_A*p_A, n_B*p_B])
print('stat', stat)
print('p-value', p_value)

# Verificação do nível de significância
alpha = 0.05
if p_value < alpha:
    print("Rejeitamos a hipótese nula. Há diferença entre valores observados e esperados na\
          conversão de 1 dia entre os dois grupos de teste.")
else:
    print("Não rejeitamos a hipótese nula. Não há diferença entre valores observados e esperados na\
          conversão de 1 dia entre os dois grupos de teste.")


stat 0.0
p-value 1.0
Não rejeitamos a hipótese nula. Não há diferença entre valores observados e esperados na          conversão de 1 dia entre os dois grupos de teste.


##### scipy.chi2_contingency

A função scipy.chi2_contingency é usada para testar a independência entre duas variáveis categóricas. Ela é usada para avaliar se duas variáveis estão relacionadas ou não. Ela aceita como entrada uma matriz de contingência, que representa o número de ocorrências em cada combinação de categorias, e retorna o estatístico de teste chi-square, o p-valor, as frequências esperadas e a matriz de contingência ajustada.

In [37]:
import scipy.stats as stats 
#from stats import binom, chisquare, chi2_contingency

#dados_chi = np.array([[conversao1_30, conversao7_30],
#             [conversao1_40, conversao7_40]])

dados_chi = np.array([[taxa1_30, taxa7_30],
             [taxa1_40, taxa7_40]])

print(dados_chi)

# Cálculo do teste chi-quadrado
stat, p, dof, expected = stats.chi2_contingency(dados_chi)

print("Estatística de teste: ", stat)
print("p-value: ", p)
print("Graus de liberdade: ", dof)
print("Valores esperados: \n", expected)

# Verificação do nível de significância
alpha = 0.05
if p < alpha:
    print("Rejeitamos a hipótese nula. Há diferença significamente estatística entre conversão de dias de jogos\
         e níveis de jogos.")
else:
    print("Não rejeitamos a hipótese nula. Não há diferença significamente estatística entre conversão de dias de jogos\
         e níveis de jogos.")


[[0.44818792 0.19020134]
 [0.44228275 0.18200004]]
Estatística de teste:  3.779447056154972
p-value:  0.051885804351360726
Graus de liberdade:  1
Valores esperados: 
 [[0.45020947 0.18817979]
 [0.4402612  0.18402159]]
Não rejeitamos a hipótese nula. Não há diferença significamente estatística entre conversão de dias de jogos         e níveis de jogos.
